In [2]:
!pip install requests
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, lower
import requests
import zipfile

In [4]:
spark = SparkSession.builder.appName("DesafioA3DATA").getOrCreate()
url = 'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2021.zip'
r = requests.get(url, verify=False)
with open("r.zip", "wb") as f:
 f.write(r.content)
 with zipfile.ZipFile("r.zip" , "r") as zip_ref:
  zip_ref.extractall("./data")

/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [8]:
df = spark.read.csv("data/Microdados do Censo da Educaç╞o Superior 2021/dados/MICRODADOS_CADASTRO_IES_2021.CSV", sep=';', header=True, inferSchema=True)
for col in df.columns:
    df = df.withColumnRenamed(col, col.lower())

In [9]:
df.printSchema()

root
 |-- nu_ano_censo: integer (nullable = true)
 |-- no_regiao_ies: string (nullable = true)
 |-- co_regiao_ies: integer (nullable = true)
 |-- no_uf_ies: string (nullable = true)
 |-- sg_uf_ies: string (nullable = true)
 |-- co_uf_ies: integer (nullable = true)
 |-- no_municipio_ies: string (nullable = true)
 |-- co_municipio_ies: integer (nullable = true)
 |-- in_capital_ies: integer (nullable = true)
 |-- no_mesorregiao_ies: string (nullable = true)
 |-- co_mesorregiao_ies: integer (nullable = true)
 |-- no_microrregiao_ies: string (nullable = true)
 |-- co_microrregiao_ies: integer (nullable = true)
 |-- tp_organizacao_academica: integer (nullable = true)
 |-- tp_categoria_administrativa: integer (nullable = true)
 |-- no_mantenedora: string (nullable = true)
 |-- co_mantenedora: integer (nullable = true)
 |-- co_ies: integer (nullable = true)
 |-- no_ies: string (nullable = true)
 |-- sg_ies: string (nullable = true)
 |-- ds_endereco_ies: string (nullable = true)
 |-- ds_numero_

In [10]:
df.show()

+------------+-------------+-------------+-----------------+---------+---------+----------------+----------------+--------------+--------------------+------------------+-------------------+-------------------+------------------------+---------------------------+--------------------+--------------+------+--------------------+------------+--------------------+----------------------+---------------------------+--------------------+----------+------------+-----------------------------+------------------------------+---------------------------+----------------------------+----------------+-----------------+-------------------+--------------------+-------------------------+--------------------------+-------------------+--------------------+--------------------+---------------------+----------------------+----------------------+--------------------+----------------------------+------------------+-------------------+------------------------+------------------+-----------------------+----------

In [ ]:
df.createOrReplaceTempView("questao1")
#Primeira questão
df1= spark.sql("""
  SELECT COUNT(DISTINCT no_municipio_ies) AS numero_de_municipios_MG FROM questao1
                WHERE sg_uf_ies ='MG'
                """)


In [12]:
df1.show()

+-----------------------+
|numero_de_municipios_MG|
+-----------------------+
|                    104|
+-----------------------+



In [13]:
df.createOrReplaceTempView("questao2")
df2= spark.sql("""
  SELECT COUNT(DISTINCT no_municipio_ies) AS numero_de_municipios_MG FROM questao2
                WHERE sg_uf_ies ='MG'
                """)

df2.show()

+-----------------------+
|numero_de_municipios_MG|
+-----------------------+
|                    104|
+-----------------------+



In [14]:
df.createOrReplaceTempView("questao3")
df3= spark.sql("""
  SELECT no_municipio_ies AS nome_municipio, SUM(qt_doc_ex_dout) AS numero_doutores FROM questao3
                WHERE sg_uf_ies ='MG'
                GROUP BY no_municipio_ies
                ORDER BY no_municipio_ies
                """)

df3.show()

+---------------+---------------+
| nome_municipio|numero_doutores|
+---------------+---------------+
|        Aimor�s|              5|
|        Alfenas|            661|
|       Almenara|              4|
|   Al�m Para�ba|             10|
|       Araguari|             55|
|          Arax�|             24|
|       Baependi|              1|
|      Barbacena|             70|
|Bar�o de Cocais|              0|
| Belo Horizonte|           6615|
|     Bert�polis|              2|
|          Betim|            192|
|  Boa Esperan�a|              1|
|   Bom Despacho|            129|
|     Brumadinho|              3|
|     Cambuquira|              3|
|     Campo Belo|              8|
|  Campos Gerais|              8|
|      Capelinha|              1|
|      Carangola|              6|
+---------------+---------------+
only showing top 20 rows



In [18]:
df1.write.parquet("primeira_questao1")
df2.write.parquet("segunda_questao2")
df3.write.parquet("terceira_questao3")


AnalysisException: [PATH_ALREADY_EXISTS] Path file:/home/jovyan/primeira_questao1 already exists. Set mode as "overwrite" to overwrite the existing path.